# Clonotype and sequence deduplication

Starting with annotated sequence data (in AbStar's `tabular` output format), reduces sequences to clonotypes and collapses dupicate clonotypes.

The [`abutils`](https://www.github.com/briney/abutils) Python package is required, and can be installed by running `pip install abutils`

*NOTE: this notebook requires the use of the Unix command line tool `sort`. Thus, it requires a Unix-based operating system to run correctly (MacOS and most flavors of Linux should be fine). Running this notebook on Windows 10 may be possible using the [Windows Subsystem for Linux](https://docs.microsoft.com/en-us/windows/wsl/about) but we have not tested this.*

In [3]:
from __future__ import print_function, division

import itertools
import multiprocessing as mp
import os
import subprocess as sp
import sys
import tempfile

from abutils.utils.jobs import monitor_mp_jobs
from abutils.utils.pipeline import list_files, make_dir
from abutils.utils.progbar import progress_bar

### years, directories and data fields

The input data (annotated sequences in [abstar's](https://github.com/briney/abstar) `tabular` format) is too large to be stored in a Github repository. A compressed archive of the data can be downloaded [**here**](https://burtonlab.s3.amazonaws.com/Collin/Recalled_Leukopaks/techrep_merged/techrep-merged_minimal_no-header.tar.gz). The data file is fairly large (about 400GB uncompressed), so make sure you have enough space before downloading. Decompressing the archive from within the `data` directory (located in the same parent directory as this notebook) will allow the code in this notebook to run without modification. If you would prefer to store the input data somewhere else, be sure to modify the `raw_input_dir` path below.

The data fields defined below correspond to the prosition in abstar's `tabular` format. If for some reason you have a differently formatted annotation file, change the field positions to suit your annotation file.

In [4]:
# years
with open('./data/years.txt') as f:
    years = sorted(f.read().split())

# directories
raw_input_dir = './data/techrep_merged/'
raw_clonotype_dir = './data/techrep-merged_vj-aa/'
make_dir(raw_clonotype_dir)
dedup_clonotype_dir = './data/dedup_techrep-merged_vj-aa/'
make_dir(dedup_clonotype_dir)
dedup_sequence_dir = './data/dedup_techrep-merged_nt-seq/'
make_dir(dedup_sequence_dir)
logfile = './data/dedup.log'

# data fields
chain_field=2
prod_field = 3
v_field = 5
j_field = 9
cdr3aa_field = 20
vdjnt_field = 22

## Deduplication (biological replicates)

In [5]:
def dedup_bioreps(files, raw_clonotype_dir, unique_clonotype_dir,
                  raw_sequence_dir, unique_sequence_dir, log_file=None):
    # set up output directories
    make_dir(raw_clonotype_dir)
    make_dir(unique_clonotype_dir)
    make_dir(raw_sequence_dir)
    make_dir(unique_sequence_dir)
    
    # process minimal output files
    for _f in files:
        print(os.path.basename(_f))
        clonotype_output_data = []
        sequence_output_data = []
        raw_clonotype_file = os.path.join(raw_clonotype_dir, os.path.basename(_f))
        unique_clonotype_file = os.path.join(unique_clonotype_dir, os.path.basename(_f))
        raw_sequence_file = os.path.join(raw_sequence_dir, os.path.basename(_f))
        unique_sequence_file = os.path.join(unique_sequence_dir, os.path.basename(_f))
        
        # collect clonotype/sequence information
        with open(_f) as f:
            for line in f:
                data = line.strip().split(',')
                if data[prod_field] != 'yes' or data[chain_field] != 'heavy':
                    continue
                v_gene = data[v_field]
                j_gene = data[j_field]
                cdr3_aa = data[cdr3aa_field]
                vdj_nt = data[vdjnt_field]
                clonotype_output_data.append(' '.join([v_gene, j_gene, cdr3_aa]))
                sequence_output_data.append(' '.join([v_gene, j_gene, vdj_nt]))
        
        # write raw clonotype info to file
        raw_clonotype_string = '\n'.join(clonotype_output_data)
        with open(raw_clonotype_file, 'w') as rf:
            rf.write(raw_clonotype_string)
        raw_clonotype_count = len(clonotype_output_data)
        print('raw clonotypes:', raw_clonotype_count)
        # collapse duplicate clonotypes (without counts)
        uniq_cmd = 'sort -u -o {} -'.format(unique_clonotype_file)
        p = sp.Popen(uniq_cmd, stdout=sp.PIPE, stderr=sp.PIPE, stdin=sp.PIPE, shell=True, encoding='utf8')
        stdout, stderr = p.communicate(input=raw_clonotype_string)
        # count the number of unique clonotypes
        wc_cmd = 'wc -l {}'.format(unique_clonotype_file)
        q = sp.Popen(wc_cmd, stdout=sp.PIPE, stderr=sp.PIPE, shell=True, encoding='utf8')
        _count, _ = q.communicate()
        unique_clonotype_count = int(_count.split()[0])
        print('unique clonotypes:', unique_clonotype_count)
        if log_file is not None:
            with open(log_file, 'a') as f:
                f.write('CLONOTYPES: {} {}\n'.format(raw_clonotype_count, unique_clonotype_count))
                
        # write raw sequence info to file
        raw_sequence_string = '\n'.join(sequence_output_data)
        with open(raw_sequence_file, 'w') as rf:
            rf.write(raw_sequence_string)
        raw_sequence_count = len(sequence_output_data)
        print('raw sequences:', raw_sequence_count)
        # collapse duplicate sequences (without counts)
        uniq_cmd = 'sort -u -o {} -'.format(unique_sequence_file)
        p = sp.Popen(uniq_cmd, stdout=sp.PIPE, stderr=sp.PIPE, stdin=sp.PIPE, shell=True, encoding='utf8')
        stdout, stderr = p.communicate(input=raw_sequence_string)
        # count the number of unique sequences
        wc_cmd = 'wc -l {}'.format(unique_sequence_file)
        q = sp.Popen(wc_cmd, stdout=sp.PIPE, stderr=sp.PIPE, shell=True, encoding='utf8')
        _count, _ = q.communicate()
        unique_sequence_count = int(_count.split()[0])
        print('unique sequences:', unique_sequence_count)
        if log_file is not None:
            with open(log_file, 'a') as f:
                f.write('SEQUENCES: {} {}\n'.format(raw_sequence_count, unique_sequence_count))
        
        print('')

In [11]:
# clear the logfile
with open(logfile, 'w') as f:
    f.write('')

# iteratively process each year
for year in years:
    print('---------')
    print(year)
    print('---------')
    print()
    with open(logfile, 'a') as f:
        f.write('#' + year + '\n')
    files = list_files('./data/techrep_merged/{}'.format(year))
    raw_clonotype_dir = './data/techrep-merged_vj-aa/{}'.format(year)
    unique_clonotype_dir = './data/dedup_techrep-merged_vj-aa/{}'.format(year)
    raw_sequence_dir = './data/techrep-merged_vdj-nt/{}'.format(year)
    unique_sequence_dir = './data/dedup_techrep-merged_vdj-nt/{}'.format(year)
    dedup_bioreps(files, raw_clonotype_dir, unique_clonotype_dir,
                  raw_sequence_dir, unique_sequence_dir, log_file=logfile)
    print('')

---------
327059-2016
---------

327059-2016_biorep1
raw clonotypes: 10398662
unique clonotypes: 4315442
raw sequences: 10398662
unique sequences: 6838434

327059-2016_biorep2
raw clonotypes: 6672566
unique clonotypes: 2065391
raw sequences: 6672566
unique sequences: 4130101

327059-2016_biorep3
raw clonotypes: 4266975
unique clonotypes: 1237575
raw sequences: 4266975
unique sequences: 2619620

327059-2016_biorep4
raw clonotypes: 2536805
unique clonotypes: 641605
raw sequences: 2536805
unique sequences: 1533250

327059-2016_biorep5
raw clonotypes: 4164872
unique clonotypes: 1176969
raw sequences: 4164872
unique sequences: 2567687

327059-2016_biorep6
raw clonotypes: 3279502
unique clonotypes: 909372
raw sequences: 3279502
unique sequences: 2018997


---------
327059-2020
---------

327059-2020_biorep1
raw clonotypes: 2270314
unique clonotypes: 813040
raw sequences: 2270314
unique sequences: 1602663

327059-2020_biorep2
raw clonotypes: 4826120
unique clonotypes: 1964478
raw sequences: 4

## Deduplication (year pools)

In the previous blocks of code, we created a unique clonotype file for each biological replicate for each year. Here, we'd like to create a single file for each year containing only unique clonotypes (regardless of which biological replicate they came from).

In [6]:
dedup_clonotype_year_pool_dir = './data/dedup_year_clonotype_pools/'
dedup_sequence_year_pool_dir = './data/dedup_year_sequence_pools/'
make_dir(dedup_clonotype_year_pool_dir)
make_dir(dedup_sequence_year_pool_dir)

First we want to create a unique clonotype file for each year that also contains the number of times we saw each clonotype (using the deduplicated biological replicates, so the clonotype count essentially tallies the number of biological replicates in which we observed each clonotype)

In [22]:
for year in years[:1]:
    print(year)
    
    # clonotypes
    input_clonotype_files = list_files(os.path.join(dedup_clonotype_dir, year))
    ofile = os.path.join(dedup_clonotype_year_pool_dir, '{}_dedup_pool_vj-aa_with-counts.txt'.format(year))
    uniq_cmd = 'cat {} | sort | uniq -c > {}'.format(' '.join(input_clonotype_files), ofile)
    c = sp.Popen(uniq_cmd, stdout=sp.PIPE, stderr=sp.PIPE, shell=True, encoding='utf8')
    stdout, stderr = c.communicate()
    
    # sequences
    input_sequence_files = list_files(os.path.join(dedup_sequence_dir, year))
    ofile = os.path.join(dedup_sequence_year_pool_dir, '{}_dedup_pool_vdj-nt_with-counts.txt'.format(year))
    uniq_cmd = 'cat {} | sort | uniq -c > {}'.format(' '.join(input_sequence_files), ofile)
    s = sp.Popen(uniq_cmd, stdout=sp.PIPE, stderr=sp.PIPE, shell=True, encoding='utf8')
    stdout, stderr = s.communicate()

327059-2016


Now the same process, but without counts:

In [27]:
for year in years:
    print(year)
        
    # clonotypes
    input_clonotype_files = list_files(os.path.join(dedup_clonotype_dir, year))
    ofile = os.path.join(dedup_clonotype_year_pool_dir, '{}_dedup_pool_vj-aa.txt'.format(year))
    uniq_cmd = 'cat {} | sort | uniq > {}'.format(' '.join(input_clonotype_files), ofile)
    c = sp.Popen(uniq_cmd, stdout=sp.PIPE, stderr=sp.PIPE, shell=True, encoding='utf8')
    stdout, stderr = c.communicate()
    
    # sequences
    input_sequence_files = list_files(os.path.join(dedup_sequence_dir, year))
    ofile = os.path.join(dedup_sequence_year_pool_dir, '{}_dedup_pool_vdj-nt.txt'.format(year))
    uniq_cmd = 'cat {} | sort | uniq > {}'.format(' '.join(input_sequence_files), ofile)
    s = sp.Popen(uniq_cmd, stdout=sp.PIPE, stderr=sp.PIPE, shell=True, encoding='utf8')
    stdout, stderr = s.communicate()

327059-2016
327059-2020
D103-2016
D103-2021


## Deduplication (cross-year pools)

Finally, we'd like to create unique clonotype files (with counts) for every groupwise combination of our 3 years. Each group can contain two or more years, meaning the total number of possible groupwise combinations is quite large. We'll use the `multiprocessing` package to parallelize the process which should speed things up substantially, although even with parallelization, this will take some time.

***NOTE:*** *The output from the following code blocks will be quite large (deduplicated clonotype files are >2TB in total, deduplicated sequence files are >20TB in total). Make sure you have sufficient storage and that the output paths below (`dedup_cross_year_clonotype_pool_dir` and `dedup_cross_year_sequence_pool_dir` are correct before starting.*

In [8]:
# directories
dedup_cross_year_clonotype_pool_dir = './data/dedup_cross-year_clonotype_pools/'
dedup_cross_year_sequence_pool_dir = './data/dedup_cross-year_sequence_pools/'
make_dir(dedup_cross_year_clonotype_pool_dir)
make_dir(dedup_cross_year_sequence_pool_dir)

# deduplicated year pool files
dedup_clonotype_year_files = [f for f in list_files(dedup_clonotype_year_pool_dir) if '_dedup_pool_vj-aa.txt' in f]
dedup_sequence_year_files = [f for f in list_files(dedup_sequence_year_pool_dir) if '_dedup_pool_vdj-nt.txt' in f]

# every possible groupwise combination of years (2 or more years per group)
year_combinations_by_size = {}
for size in range(2, 5):
    year_combinations_by_size[size] = [sorted(c) for c in itertools.combinations(years, size)]

In [10]:
def dedup_cross_year_pool(years, dedup_year_files, output_dir, file_end, temp_dir='/tmp/'):
    files = sorted(list(set([f for f in dedup_year_files if os.path.basename(f).split('_')[0] in years])))
    output_file = os.path.join(output_dir, '{}{}'.format('-'.join(years), file_end))
    uniq_cmd = 'cat {} | sort -T {} | uniq -c > {}'.format(' '.join(files), temp_dir, output_file)
    p = sp.Popen(uniq_cmd, stdout=sp.PIPE, stderr=sp.PIPE, shell=True, encoding='utf8')
    stdout, stderr = p.communicate()

### Clonotypes

In [14]:
p = mp.Pool(maxtasksperchild=1)

for size in sorted(year_combinations_by_size.keys())[:1]:
    year_combinations = year_combinations_by_size[size]
    async_results = []
    print('{}-year pools:'.format(size))
    progress_bar(0, len(year_combinations))
    for sub_comb in year_combinations:
        files = sorted(list(set([f for f in dedup_clonotype_year_files if os.path.basename(f).split('_')[0] in sub_comb])))
        async_results.append(p.apply_async(dedup_cross_year_pool,
                                           args=(sub_comb, files, dedup_cross_year_clonotype_pool_dir, '_dedup_pool_vj-aa_with-counts.txt')))
    monitor_mp_jobs(async_results)
    print('\n')

p.close()
p.join()

2-year pools:
(6/6) ||||||||||||||||||||||||||||||||||||||||||||||||||||  100%  
(6/6) ||||||||||||||||||||||||||||||||||||||||||||||||||||  100%  





### Sequences

Just one more warning that the following code block will produce a very large amount of data (>20TB) and will take many hours to run even on a fairly robust server (an `m4.16xlarge` AWS EC2 instance, for example).

In [13]:
p = mp.Pool(maxtasksperchild=1)

for size in sorted(year_combinations_by_size.keys())[:1]:
    year_combinations = year_combinations_by_size[size]
    async_results = []
    print('{}-year pools:'.format(size))
    progress_bar(0, len(year_combinations))
    for sub_comb in year_combinations:
        files = sorted(list(set([f for f in dedup_sequence_year_files if os.path.basename(f).split('_')[0] in sub_comb])))
        async_results.append(p.apply_async(dedup_cross_year_pool,
                                           args=(sub_comb, files, dedup_cross_year_sequence_pool_dir, '_dedup_pool_vdj-nt_with-counts.txt')))
    monitor_mp_jobs(async_results)
    print('\n')

p.close()
p.join()

2-year pools:
(6/6) ||||||||||||||||||||||||||||||||||||||||||||||||||||  100%  
(6/6) ||||||||||||||||||||||||||||||||||||||||||||||||||||  100%  



